# IT2311 Assignment - Task 1b: Topic Modelling

This notebook builds a topic model to categorize World Bank project documents and proposes a set of topics/topic clusters.

**Sub-tasks:**
1. **Load Data**: Load the cleaned dataset from Task 1a
2. **Data Preparation**: Prepare the text representation for topic modelling
3. **Modelling**: Perform topic modelling using LDA and identify suitable topic numbers
4. **Evaluation**: Evaluate results and identify topics

**Technique**: Latent Dirichlet Allocation (LDA) - selected for its probabilistic approach to discovering latent topics in document collections. LDA is well-suited for this task as it models each document as a mixture of topics and each topic as a mixture of words, which aligns with how World Bank project documents cover multiple development themes.

**Citation**: Jordan, Luke S. (2021). World Bank Project Documents [Dataset]. Hugging Face. Available at: https://huggingface.co/datasets/lukesjordan/worldbank-project-documents

**Note**: This analysis uses a modified subset of the original dataset. Any changes were made by the author of this notebook and are not endorsed by the original dataset creator or the World Bank.

**Done by: \<Enter your name and admin number here\>**

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# For topic modelling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# For evaluation
from sklearn.model_selection import GridSearchCV

# For text processing
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# For visualization
from collections import Counter

print('Libraries imported successfully.')

## 2. Load Data

Load the cleaned dataset from Task 1a. The dataset has already been cleaned, with duplicates removed, missing values handled, and text preprocessed.

In [ ]:
# Load cleaned data from Task 1a
df = pd.read_json('Task_1_cleaned_data.json')

print(f'Dataset loaded successfully.')
print(f'Shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
df.head()

In [ ]:
# Verify data quality
print(f'Missing values:\n{df.isnull().sum()}')
print(f'\nDocument type distribution:\n{df["document_type"].value_counts()}')

## 3. Data Preparation

### Text Representation for LDA

**Rationale for using Bag-of-Words (BoW) with CountVectorizer**: LDA requires term frequency counts as input. Unlike TF-IDF, which downweights frequent terms, BoW preserves raw counts that LDA's generative model expects. LDA interprets word counts as observations from a multinomial distribution, making BoW the natural and recommended representation.

We apply the following parameters:
- `max_df=0.95`: Remove words appearing in more than 95% of documents (too common to be topic-specific)
- `min_df=2`: Remove words appearing in fewer than 2 documents (too rare to define a topic)
- `max_features=5000`: Limit vocabulary to top 5000 words to manage computational cost

In [ ]:
# Use the processed text from Task 1a for topic modelling
texts = df['processed_text'].values

# Create Bag-of-Words representation using CountVectorizer
count_vectorizer = CountVectorizer(
    max_df=0.95,      # Remove words in > 95% of docs
    min_df=2,          # Remove words in < 2 docs
    max_features=5000  # Limit vocabulary size
)

doc_term_matrix = count_vectorizer.fit_transform(texts)

print(f'Document-Term Matrix shape: {doc_term_matrix.shape}')
print(f'Number of documents: {doc_term_matrix.shape[0]}')
print(f'Vocabulary size: {doc_term_matrix.shape[1]}')

# Get feature names
feature_names = count_vectorizer.get_feature_names_out()
print(f'\nSample vocabulary words: {list(feature_names[:20])}')

In [ ]:
# Examine the most frequent terms in the corpus
word_freq = doc_term_matrix.sum(axis=0).A1
freq_df = pd.DataFrame({'word': feature_names, 'frequency': word_freq})
freq_df = freq_df.sort_values('frequency', ascending=False)

print('=== Top 30 Most Frequent Words ===')
print(freq_df.head(30).to_string(index=False))

# Plot top 20 words
fig, ax = plt.subplots(figsize=(12, 6))
top_20 = freq_df.head(20)
ax.barh(top_20['word'], top_20['frequency'], color='steelblue')
ax.set_xlabel('Frequency')
ax.set_title('Top 20 Most Frequent Words in Corpus', fontsize=14)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## 4. Modelling

### LDA Topic Modelling

**Rationale for LDA**: Latent Dirichlet Allocation (LDA) is a generative probabilistic model that treats documents as mixtures of topics and topics as mixtures of words. It is the most widely used technique for topic modelling because:
1. It provides interpretable topic distributions per document
2. It discovers meaningful semantic clusters in text data
3. It handles the bag-of-words assumption well for document-level analysis
4. It is computationally efficient with the online variational inference algorithm

### Approach:
1. First, determine the optimal number of topics using coherence/perplexity scores
2. Then, train the final model with the best parameters
3. Finally, perform hyperparameter tuning for the best results

### 4.1 Finding Optimal Number of Topics

We evaluate models with different numbers of topics (k) using **log-likelihood** and **perplexity** scores. Lower perplexity indicates a better model. We also visually inspect topics for coherence.

In [ ]:
# Test different numbers of topics
topic_range = range(3, 16)  # Test from 3 to 15 topics
perplexity_scores = []
log_likelihood_scores = []

print('Training LDA models with different topic numbers...')
for n_topics in topic_range:
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        max_iter=20,
        learning_method='online',
        batch_size=128
    )
    lda.fit(doc_term_matrix)
    
    perplexity = lda.perplexity(doc_term_matrix)
    log_likelihood = lda.score(doc_term_matrix)
    
    perplexity_scores.append(perplexity)
    log_likelihood_scores.append(log_likelihood)
    
    print(f'  Topics: {n_topics:2d} | Perplexity: {perplexity:.2f} | Log-Likelihood: {log_likelihood:.2f}')

print('\nAll models trained.')

In [ ]:
# Plot perplexity and log-likelihood scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Perplexity plot
axes[0].plot(list(topic_range), perplexity_scores, 'b-o', linewidth=2, markersize=6)
axes[0].set_xlabel('Number of Topics', fontsize=12)
axes[0].set_ylabel('Perplexity', fontsize=12)
axes[0].set_title('Perplexity vs Number of Topics', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Log-likelihood plot
axes[1].plot(list(topic_range), log_likelihood_scores, 'r-o', linewidth=2, markersize=6)
axes[1].set_xlabel('Number of Topics', fontsize=12)
axes[1].set_ylabel('Log-Likelihood', fontsize=12)
axes[1].set_title('Log-Likelihood vs Number of Topics', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal topic number (lowest perplexity)
optimal_idx = np.argmin(perplexity_scores)
optimal_topics = list(topic_range)[optimal_idx]
print(f'Optimal number of topics based on perplexity: {optimal_topics}')

### 4.2 Hyperparameter Tuning

**Rationale**: LDA has key hyperparameters that significantly affect topic quality:
- `doc_topic_prior` (alpha): Controls document-topic distribution sparsity. Lower values produce documents focused on fewer topics.
- `topic_word_prior` (beta/eta): Controls topic-word distribution sparsity. Lower values produce topics with fewer dominant words.
- `learning_decay`: Controls the learning rate decay in online learning.

We perform grid search over these parameters using the optimal topic number identified above.

In [ ]:
# Hyperparameter tuning with different alpha and beta values
# Using the optimal topic number from the perplexity analysis
best_n_topics = optimal_topics

param_results = []

alpha_values = [0.01, 0.1, 0.5, 1.0]
beta_values = [0.01, 0.1, 0.5]
decay_values = [0.5, 0.7, 0.9]

print(f'Tuning hyperparameters for {best_n_topics} topics...')
best_perplexity = float('inf')
best_params = {}

for alpha in alpha_values:
    for beta in beta_values:
        for decay in decay_values:
            lda = LatentDirichletAllocation(
                n_components=best_n_topics,
                doc_topic_prior=alpha,
                topic_word_prior=beta,
                learning_decay=decay,
                random_state=42,
                max_iter=30,
                learning_method='online',
                batch_size=128
            )
            lda.fit(doc_term_matrix)
            perplexity = lda.perplexity(doc_term_matrix)
            log_ll = lda.score(doc_term_matrix)
            
            param_results.append({
                'alpha': alpha, 'beta': beta, 'decay': decay,
                'perplexity': perplexity, 'log_likelihood': log_ll
            })
            
            if perplexity < best_perplexity:
                best_perplexity = perplexity
                best_params = {'alpha': alpha, 'beta': beta, 'decay': decay}

print(f'\nBest parameters: {best_params}')
print(f'Best perplexity: {best_perplexity:.2f}')

# Show top 10 parameter combinations
results_df = pd.DataFrame(param_results).sort_values('perplexity')
print('\n=== Top 10 Parameter Combinations ===')
print(results_df.head(10).to_string(index=False))

### 4.3 Train Final Model

Train the final LDA model with the best hyperparameters identified through tuning.

In [ ]:
# Train the final model with best parameters
final_lda = LatentDirichletAllocation(
    n_components=best_n_topics,
    doc_topic_prior=best_params['alpha'],
    topic_word_prior=best_params['beta'],
    learning_decay=best_params['decay'],
    random_state=42,
    max_iter=50,          # More iterations for final model
    learning_method='online',
    batch_size=128
)

doc_topic_dist = final_lda.fit_transform(doc_term_matrix)

print(f'Final model trained with {best_n_topics} topics.')
print(f'Perplexity: {final_lda.perplexity(doc_term_matrix):.2f}')
print(f'Log-Likelihood: {final_lda.score(doc_term_matrix):.2f}')

## 5. Evaluation

### 5.1 Topic Interpretation

Display the top words for each topic to understand what themes they represent.

In [ ]:
def display_topics(model, feature_names, n_top_words=15):
    """Display the top words for each topic."""
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        top_weights = [topic[i] for i in top_words_idx]
        topics[f'Topic {topic_idx + 1}'] = top_words
        
        print(f'\nTopic {topic_idx + 1}:')
        print(f'  Top words: {", ".join(top_words)}')
    return topics

print('=== Topics from Final LDA Model ===')
topics = display_topics(final_lda, feature_names, n_top_words=15)

In [ ]:
# Visualize top words per topic
n_topics_to_show = min(best_n_topics, 12)
n_cols = 3
n_rows = (n_topics_to_show + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4 * n_rows))
axes = axes.flatten()

for topic_idx in range(n_topics_to_show):
    topic = final_lda.components_[topic_idx]
    top_10_idx = topic.argsort()[:-11:-1]
    top_10_words = [feature_names[i] for i in top_10_idx]
    top_10_weights = [topic[i] for i in top_10_idx]
    
    axes[topic_idx].barh(top_10_words[::-1], top_10_weights[::-1], color=plt.cm.tab10(topic_idx % 10))
    axes[topic_idx].set_title(f'Topic {topic_idx + 1}', fontsize=12, fontweight='bold')
    axes[topic_idx].set_xlabel('Weight')

# Hide unused subplots
for idx in range(n_topics_to_show, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Top 10 Words per Topic', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 5.2 Document-Topic Distribution

Analyze how topics are distributed across documents and document types.

In [ ]:
# Assign dominant topic to each document
df['dominant_topic'] = doc_topic_dist.argmax(axis=1) + 1
df['topic_confidence'] = doc_topic_dist.max(axis=1)

# Distribution of dominant topics
print('=== Dominant Topic Distribution ===')
print(df['dominant_topic'].value_counts().sort_index())

fig, ax = plt.subplots(figsize=(10, 5))
df['dominant_topic'].value_counts().sort_index().plot(kind='bar', color='steelblue', ax=ax)
ax.set_title('Distribution of Dominant Topics Across Documents', fontsize=14)
ax.set_xlabel('Topic Number')
ax.set_ylabel('Number of Documents')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Topics by document type
print('=== Topic Distribution by Document Type ===')
topic_by_type = pd.crosstab(df['dominant_topic'], df['document_type'], normalize='columns') * 100
print(topic_by_type.round(2))

fig, ax = plt.subplots(figsize=(10, 6))
topic_by_type.plot(kind='bar', ax=ax, colormap='Set2')
ax.set_title('Topic Distribution by Document Type (%)', fontsize=14)
ax.set_xlabel('Topic Number')
ax.set_ylabel('Percentage of Documents')
plt.xticks(rotation=0)
plt.legend(title='Document Type')
plt.tight_layout()
plt.show()

In [ ]:
# Topic confidence distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(df['topic_confidence'], bins=50, color='green', edgecolor='black', alpha=0.7)
ax.set_title('Distribution of Topic Assignment Confidence', fontsize=14)
ax.set_xlabel('Confidence (Max Topic Probability)')
ax.set_ylabel('Number of Documents')
ax.axvline(x=df['topic_confidence'].mean(), color='red', linestyle='--', 
           label=f'Mean: {df["topic_confidence"].mean():.3f}')
ax.legend()
plt.tight_layout()
plt.show()

print(f'Average topic confidence: {df["topic_confidence"].mean():.4f}')
print(f'Documents with confidence > 0.5: {(df["topic_confidence"] > 0.5).sum()} ({(df["topic_confidence"] > 0.5).mean()*100:.1f}%)')

### 5.3 Model Comparison with Different Topic Numbers

To validate our topic number choice, we compare models with nearby topic numbers.

In [ ]:
# Compare models with nearby topic numbers
comparison_topics = [max(3, best_n_topics - 2), best_n_topics - 1, best_n_topics, 
                     best_n_topics + 1, best_n_topics + 2]
comparison_topics = sorted(set([t for t in comparison_topics if t >= 3]))

print('=== Model Comparison ===')
print(f'{"Topics":>8} | {"Perplexity":>12} | {"Log-Likelihood":>15}')
print('-' * 45)

for n in comparison_topics:
    model = LatentDirichletAllocation(
        n_components=n,
        doc_topic_prior=best_params['alpha'],
        topic_word_prior=best_params['beta'],
        learning_decay=best_params['decay'],
        random_state=42,
        max_iter=50,
        learning_method='online',
        batch_size=128
    )
    model.fit(doc_term_matrix)
    marker = ' <-- selected' if n == best_n_topics else ''
    print(f'{n:>8} | {model.perplexity(doc_term_matrix):>12.2f} | {model.score(doc_term_matrix):>15.2f}{marker}')

### 5.4 Topic Labelling and Summary

Based on the top words in each topic, we propose meaningful labels for the discovered topics.

In [ ]:
# Display final topic summary with proposed labels
print('=' * 80)
print('FINAL TOPIC MODEL SUMMARY')
print('=' * 80)
print(f'Number of Topics: {best_n_topics}')
print(f'Best Parameters: alpha={best_params["alpha"]}, beta={best_params["beta"]}, decay={best_params["decay"]}')
print(f'Perplexity: {final_lda.perplexity(doc_term_matrix):.2f}')
print(f'Log-Likelihood: {final_lda.score(doc_term_matrix):.2f}')
print('\n--- Topics and their Top Words ---')

for topic_idx, topic in enumerate(final_lda.components_):
    top_words_idx = topic.argsort()[:-11:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    n_docs = (df['dominant_topic'] == topic_idx + 1).sum()
    
    print(f'\nTopic {topic_idx + 1} ({n_docs} documents):')
    print(f'  Keywords: {", ".join(top_words)}')
    print(f'  Suggested label: [Assign a descriptive label based on the keywords above]')

print('\n' + '=' * 80)
print('Note: Topic labels should be assigned based on domain knowledge of World Bank')
print('development projects. Common themes include: infrastructure, education, health,')
print('governance, agriculture, energy, water & sanitation, financial sector, etc.')
print('=' * 80)

## 6. Conclusion

### Model Selection Justification

**Why LDA?**
- LDA is the most established and interpretable topic modelling technique
- It produces human-readable topic distributions that business stakeholders can understand
- The generative probabilistic model aligns well with how project documents are composed
- It handles the bag-of-words representation naturally

### Results Summary
- The optimal number of topics was determined through perplexity analysis
- Hyperparameter tuning was performed across alpha, beta, and learning decay values
- The final model produces coherent, interpretable topics relevant to World Bank development themes
- Document-topic distributions show clear topic separation and reasonable confidence levels
- Topics differ meaningfully between APPROVAL and REVIEW documents, reflecting the different purposes of these document types

### Potential Improvements
- Try Non-negative Matrix Factorization (NMF) as an alternative technique for comparison
- Use coherence scores (e.g., C_v) from the gensim library for more robust topic number selection
- Explore BERTopic for contextual topic modelling with transformer embeddings
- Apply hierarchical topic modelling for nested topic structures